In [4]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Load data
df = pd.read_csv("example_data.csv", parse_dates=['time'])

# List of parameters for selection
PARAMS = {
    "temperature": "Temperature (°C)",
    "salinity": "Salinity (PSU)",
    "co2": "CO₂ (ppm)"
}

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Sailing Ship Data Dashboard"),
    dcc.Dropdown(
        id='parameter-dropdown',
        options=[{"label": v, "value": k} for k, v in PARAMS.items()],
        value="temperature",
        clearable=False,
        style={"width": "300px"}
    ),
    dcc.Graph(id='time-series-plot'),
    dcc.Graph(id='route-map')
])

@app.callback(
    [Output('time-series-plot', 'figure'),
     Output('route-map', 'figure')],
    [Input('parameter-dropdown', 'value')]
)
def update_dashboard(selected_param):
    # Time series plot
    fig_time = px.line(
        df,
        x="time",
        y=selected_param,
        title=f"{PARAMS[selected_param]} over Time",
        markers=True,
        labels={"time": "Time", selected_param: PARAMS[selected_param]}
    )

    # Route map with color gradient
    fig_map = px.scatter_geo(
        df,
        lat="latitude",
        lon="longitude",
        color=selected_param,
        color_continuous_scale="Viridis",
        title=f"Route Map Colored by {PARAMS[selected_param]}",
        labels={selected_param: PARAMS[selected_param]}
    )
    # Draw lines between points
    fig_map.update_traces(mode="lines+markers", marker=dict(size=8))
    fig_map.update_geos(
        showcountries=True, showcoastlines=True, projection_type="natural earth"
    )
    fig_map.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

    return fig_time, fig_map

if __name__ == '__main__':
    app.run(debug=True)


ValueError: Missing column provided to 'parse_dates': 'time'